# [Decision Trees](https://www.youtube.com/watch?v=Bqi7EFFvNOg&list=PLqnslRFeH2Upcrywf-u2etjdxxkL8nl7E&index=9)
A decision tree tries to build a tree to split our data with the best separation of classes, and is also the basis for the random forest model which is covered in random_forest.ipynb. 

In this example, say we want to predict whether a person is walking or taking the bus to work - we have two features, `Rain` and `Time` and the response `Walk` which is classified as `No` or `Yes`. To build a tree that splits this data, we have all of the samples in the root node, and ask a question such as if it is raining, and then split the samples accordingly. We can then further split these samples into more leaves. 



### Entropy 
We will use entropy as the criterion for how to split regions. This is a measurement of uncertainty. 

$E = -\overset{K}{\underset{k=1}{\Sigma}}p(X)\log_2 p(X)$

$p(X) = \frac{\#x}{n}$

The formula is the sum of $p(X)$ times the log of $p(X)$. So we are looking at the product of $p(X)$ and $\log p(X)$ for each class. This entropy gives us a value between 0 and 1, where 1 is the worst case (a 50/50 split of classes so no prediction can be made) and 0 is the best case (the region contains only one class). 

Using this dataset as an example, we have ten samples where 5 are 0 and 5 are 1: 

$S = [0,0,0,0,0,1,1,1,1,1]$

Our entropy is: 

$E = -0.5 \cdot (-1) - 0.5 \cdot (-1) = 1 $

which is the worst case. 

### Information gain
Information gain is a measurement of the reduction in entropy from splitting the regions. If we were to begin by splitting based on rain, we would have: 

$S = [0,0,0,0,0,1,1,1,1,1], S_1 = [0,0,1,1,1,1,1], S_2 = [0,0,0]$

The formula for information gain is given as: 

$E(\text{parent}) - [\text{weighted average} \cdot E(\text{children})]$

and in this example would look like: 

$E(S) - [(7/10) * E(S_1) + (3/10) * E(S_2)]$

$1 - [(7/10) \times 0.863 + (3/10) * 0] = 0.395$

We can use the information gain to perform a greedy search (going over all possible features and all possible feature thresholds) and selecting the best feature and threshold. 

### Algorithm 
We are going to have two parts to our decision tree classifier: first, we will build the tree by starting with the full set of data, and creating subsets based on a greedy search of information gain, and build this tree with recursive binary splitting with some kind of stopping criteria to prevent growth. We will then use the tree to predict by traversing the tree, checking each feature and returning a classification upon reaching a terminal node. 

In [21]:
import numpy as np
from collections import Counter
from sklearn import datasets 
from sklearn.model_selection import train_test_split
seed = 2

In [22]:
def entropy(y):
    ' Takes in a vector of class labels and outputs the entropy ' 
    hist = np.bincount(y) # Use a histogram to count number of occurrences of all labels
    ps = hist / len(y)    # p(X)
    return -np.sum([p * np.log2(p) for p in ps if p > 0]) # Return entropy

In [23]:
class Node: 
    def __init__(self, feature=None, threshold=None, left=None, right=None, *,value=None):
        self.feature   = feature
        self.threshold = threshold 
        self.left      = left 
        self.right     = right 
        self.value     = value

    def is_leaf(self):
        ' Determines if node is a leaf node (if it has value) '
        return self.value is not None
        

In [34]:
class DecisionTree: 
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None): 
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None 

    def fit(self, X, y): 
        ' Takes in training data x and labels y to grow tree '
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        ' Traverses tree to make a prediction '
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape  # Number of samples and features
        n_labels = len(np.unique(y))     # Number of different labels

        # Stopping criteria 
        if (depth >= self.max_depth 
            or n_labels == 1
            or n_samples < self.min_samples_split): 
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # If not met stopping criteria
        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # Greedy search
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left  = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1) # We want only the left idxs but all of the features
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right) # Returns a node with the best feature, best threshold, and the left and right children

    def _most_common_label(self, y): 
        ' Takes a vector of class labels and returns the most common class '
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0] # Returns the most common label
        return most_common

    def _best_criteria(self, X, y, feat_idxs):
        ' Calculate information gain for each feature '
        best_gain = -1
        split_idx, split_thresh = None, None 
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column) 
            for threshold in thresholds: 
                gain = self._information_gain(y, X_column, threshold) 

                if gain > best_gain: 
                    best_gain = gain 
                    split_idx = feat_idx 
                    split_thresh = threshold
        
        return split_idx, split_thresh 

    def _information_gain(self, y, X_column, split_thresh):
        ' Computes information gain '
        # Parent entropy 
        parent_entropy = entropy(y)

        # Generate split 
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # Weighted average of child entropies 
        n = len(y) 
        n_left, n_right = len(left_idxs), len(right_idxs) 
        e_left, e_right = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        # Return information gain
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs  = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node): 
        ' Gets a sample and a start node and traverses the tree '
        if node.is_leaf():
            return node.value # If we are at a leaf, we have a value 

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left) # Go to the left
        
        return self._traverse_tree(x, node.right)

### Using our decision tree
We can use this implementation on the breast cancer dataset from SciKit-Learn. 

In [35]:
def accuracy(y_true, y_pred): 
    accuracy = np.sum(y_true == y_pred) / len(y_true) 
    return accuracy 

data = datasets.load_breast_cancer() 
X = data.data 
y = data.target 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

tree = DecisionTree(max_depth=10)
tree.fit(X_train, y_train) 

y_pred = tree.predict(X_test) 
acc = accuracy(y_test, y_pred)

print(f"Accuracy is: {acc}")

Accuracy is: 0.8947368421052632


### Extensions
Would be good to have tree pruning

Also want to be able to visualise the tree, see what it's doing rather than be a black box accuracy